In [114]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric

ee.Initialize()

In [116]:
def func1(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    #s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_OLI_B1,
            'B2':PredefinedWavelengths.LANDSAT_OLI_B2,
            'B3':PredefinedWavelengths.LANDSAT_OLI_B3,
            'B4':PredefinedWavelengths.LANDSAT_OLI_B4,
            'B5':PredefinedWavelengths.LANDSAT_OLI_B5,
            'B6':PredefinedWavelengths.LANDSAT_OLI_B6,
            'B7':PredefinedWavelengths.LANDSAT_OLI_B7,
            'B8':PredefinedWavelengths.LANDSAT_OLI_B8,
            'B9':PredefinedWavelengths.LANDSAT_OLI_B9,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_BAND = {
            'B1':ESUN_L8[0],
            'B2':ESUN_L8[1],
            'B3':ESUN_L8[2],
            'B4':ESUN_L8[3],
            'B5':ESUN_L8[4],
            'B6':ESUN_L8[5],
            'B7':ESUN_L8[6],
            'B8':ESUN_L8[7],
            'B9':ESUN_L8[8],
            }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    ca  = surface_reflectance('B1')
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B5')
    swir1 = surface_reflectance('B6')
    swir2 = surface_reflectance('B7')
    
    ref = img.select('BQA').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = ref.copyProperties(img).set({              
                  'AC_date':dateString,
                  'AC_aerosol_optical_thickness':aot,
                  'AC_water_vapour':h2o,
                  'AC_version':'py6S',
                  'AC_contact':'ndminhhus@gmail.com',
                  'AC_ozone':o3})
    

    # define YOUR assetID 
    # in my case it was something like this..
    #assetID = 'users/samsammurphy/shared/sentinel2/6S/ESRIN_'+dateString
    #assetID = 'users/ndminhhus/eLEAF/nt/s2_SIAC/'+fname,
    # # export
    fname = ee.String(img.get('system:index')).getInfo()
    export = ee.batch.Export.image.toAsset(\
        image=ref,
        description= 'L8_BOA_'+fname,
        assetId = 'users/ndminhhus/eLEAF/nt/L8_py6S/'+'L8_BOA'+fname,
        region = region,
        scale = 30,
        maxPixels = 1e13)

    # # uncomment to run the export
    export.start()
    print('exporting ' +fname + '--->done')

In [120]:
startDate = ee.Date('2018-01-01')
endDate = ee.Date('2020-01-01')
geom = ee.Geometry.Point(108.91220182000018,11.700863529688942)# Ninh Thuan region
region = geom.buffer(60000).bounds().getInfo()['coordinates']

In [121]:
# The Landsat 8 image collection
L8_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate).sort('CLOUD_COVER')

In [122]:
print(L8_col.size().getInfo())

28


In [91]:
L8_list = L8_col.toList(L8_col.size().getInfo())
img1 = ee.Image(L8_list.get(9))

In [92]:
toa = img1
boa = ee.Image(func1(toa))

In [124]:
print(boa.getInfo()['properties']['AC_date'])

KeyError: 'AC_date'

In [106]:
from IPython.display import display, Image

channels = ['B4','B3','B2']

original = Image(url=toa.select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.25
                }))

corrected = Image(url=ee.Image(boa).select(channels).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.25
                }))

display(original, corrected)

In [123]:
col_length = L8_col.size().getInfo()
#print(col_length)

for i in range(0,col_length):
    #print(i)
    list = L8_col.toList(col_length)
    img = ee.Image(list.get(i))
    func1(img)


exporting LC08_123052_20190614--->done
exporting LC08_123052_20181017--->done
exporting LC08_123052_20190310--->done
exporting LC08_123052_20180830--->done
exporting LC08_123052_20180424--->done
exporting LC08_123052_20180510--->done
exporting LC08_123052_20181102--->done
exporting LC08_123052_20180219--->done
exporting LC08_123052_20180323--->done
exporting LC08_123052_20180307--->done
exporting LC08_123052_20190529--->done
exporting LC08_123052_20181204--->done
exporting LC08_123052_20181220--->done
exporting LC08_123052_20190513--->done
exporting LC08_123052_20190206--->done
exporting LC08_123052_20190222--->done
exporting LC08_123052_20190105--->done
exporting LC08_123052_20190326--->done
exporting LC08_123052_20190427--->done
exporting LC08_123052_20180526--->done
exporting LC08_123052_20190411--->done
exporting LC08_123052_20190121--->done
exporting LC08_123052_20180102--->done
exporting LC08_123052_20180203--->done
exporting LC08_123052_20180627--->done
exporting LC08_123052_201